# Dataset __Bioresponse__

In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
data = pd.read_csv('bioresponse.csv')

### EDA

In [9]:
print(data.head())
print(data.info())
print(data.describe())

print(data.dtypes)

         D1        D2    D3   D4        D5        D6        D7        D8  \
0  0.000000  0.497009  0.10  0.0  0.132956  0.678031  0.273166  0.585445   
1  0.366667  0.606291  0.05  0.0  0.111209  0.803455  0.106105  0.411754   
2  0.033300  0.480124  0.00  0.0  0.209791  0.610350  0.356453  0.517720   
3  0.000000  0.538825  0.00  0.5  0.196344  0.724230  0.235606  0.288764   
4  0.100000  0.517794  0.00  0.0  0.494734  0.781422  0.154361  0.303809   

         D9       D10  ...  D1768  D1769  D1770  D1771  D1772  D1773  D1774  \
0  0.743663  0.243144  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1  0.836582  0.106480  ...    1.0    1.0    1.0    0.0    1.0    0.0    0.0   
2  0.679051  0.352308  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3  0.805110  0.208989  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4  0.812646  0.125177  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

   D1775  D1776  target  
0    0.0    0.0       1  
1    1.0    0.0 

Each row in this data set represents a molecule. The first column contains experimental data describing an actual biological response; the molecule was seen to elicit this response (1), or not (0). The remaining columns represent molecular descriptors (d1 through d1776), these are calculated properties that can capture some of the characteristics of the molecule - for example size, shape, or elemental constitution. The "target" column is the biological response.

In [10]:
X = data.drop(columns=['target']) # Input features (molecular descriptors)
Y = data['target'] # Target variable (biological response)

### Data Visualization

In [11]:
# Target value distribution
def target_distribution(data):
    plt.figure(figsize=(6, 4))
    sns.countplot(x='target', data=data, palette='pastel')
    plt.title('Distribution of the target value')
    plt.xlabel('Target Variable')
    plt.ylabel('Count')
    plt.show()

# Boxplots for the first N descriptors
def firstN_descriptors(data, num):
    plt.figure(figsize=(12, 8))
    sns.boxplot(data=data.iloc[:, 1:num])
    plt.title('Boxplot of Molecular Descriptors (d1-d10)')
    plt.xlabel('Descriptor')
    plt.ylabel('Value')
    plt.show()

# Visualisation of the relationship between the first molecular descriptor (X1) and the target variable
def descriptor_target_relationship(data, idx):
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=data['target'], y=data.iloc[:, idx], color='lightgreen')
    plt.title('Relationship between the first descriptor and Target Variable')
    plt.xlabel('Target Variable')
    plt.ylabel('X')
    plt.show()

def heatmaps_corr(data):
    X = data.drop(columns=['target']) # Input features (molecular descriptors)
    Y = data['target'] # Target variable (biological response)

    correlation_X = X.corr()  # Correlation among molecular descriptors
    correlation_Y = X.apply(lambda x: x.corr(Y))  # Correlation between each molecular descriptor and the target variable

    # Heatmap for correlation among molecular descriptors
    sns.heatmap(correlation_X, cmap='coolwarm', annot=False, ax=axes[0])
    axes[0].set_title('Correlation Heatmap - Molecular Descriptors')
    axes[0].set_xlabel('Molecular Descriptors')
    axes[0].set_ylabel('Molecular Descriptors')

    # Heatmap for correlation between molecular descriptors and target variable
    sns.heatmap(correlation_Y.to_frame().transpose(), cmap='coolwarm', annot=True, fmt=".2f", ax=axes[1])
    axes[1].set_title('Correlation Heatmap - Molecular Descriptors vs. Target Variable')
    axes[1].set_xlabel('Molecular Descriptors')
    axes[1].set_ylabel('Target Variable (Y)')

    plt.tight_layout()
    plt.show()


## Feature selection

#### Merging descriptors with similar correlation

In [12]:
# correlation_threshold = treshold for correlation to merge descriptors (bigger corr => merget descriptors) 
def merge_descriptors(data, correlation_threshold = 0.5):
    correlation_matrix = data.corr()
    merged_descriptors = set()

    # Iterate over the correlation matrix to identify pairs of descriptors with similar correlation
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) >= correlation_threshold:
                # Add correlated descriptors to the set
                merged_descriptors.add((correlation_matrix.columns[i], correlation_matrix.columns[j]))

    merged_data = data.copy()

    # Merge descriptors
    for descriptor_pair in merged_descriptors:
        # Check if both descriptors exist in the dataset
        if all(descriptor in merged_data.columns for descriptor in descriptor_pair):
            merged_descriptor_name = '_'.join(descriptor_pair)
            merged_data[merged_descriptor_name] = (data[descriptor_pair[0]] + data[descriptor_pair[1]]) / 2
            merged_data.drop(list(descriptor_pair), axis=1, inplace=True)

    print("Information about Merged DataFrame:")
    print(merged_data.info())

    return merged_data

C:\Users\matul\AppData\Local\Temp\ipykernel_24016\639554792.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data[merged_descriptor_name] = (data[descriptor_pair[0]] + data[descriptor_pair[1]]) / 2
C:\Users\matul\AppData\Local\Temp\ipykernel_24016\639554792.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data[merged_descriptor_name] = (data[descriptor_pair[0]] + data[descriptor_pair[1]]) / 2
C:\Users\matul\AppData\Local\Temp\ipykernel_24016\639554792.py:21: PerformanceWarning: DataFrame is highly fragmente

Information about Merged DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3751 entries, 0 to 3750
Columns: 1140 entries, D21 to D326_D375
dtypes: float64(1139), int64(1)
memory usage: 32.6 MB
None


C:\Users\matul\AppData\Local\Temp\ipykernel_24016\639554792.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data[merged_descriptor_name] = (data[descriptor_pair[0]] + data[descriptor_pair[1]]) / 2
C:\Users\matul\AppData\Local\Temp\ipykernel_24016\639554792.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_data[merged_descriptor_name] = (data[descriptor_pair[0]] + data[descriptor_pair[1]]) / 2
C:\Users\matul\AppData\Local\Temp\ipykernel_24016\639554792.py:21: PerformanceWarning: DataFrame is highly fragmente

#### Linear correlation

In [14]:
def correlation_selection_original(data, correlation_threshold = 0.2):
    selected_features = correlation_Y[correlation_Y >= correlation_threshold].index.tolist()

    selected_data = data[selected_features]

    print("Selected Features from original data:")
    print(selected_features)
    
    return pd.DataFrame(data=data[selected_features + ['target']])

def correlation_selection_merged(data, correlation_threshold = 0.2):
    correlation_matrix = data.corr()

    correlation_threshold_m = 0.5  # Adjust as needed

    merged_descriptors = set()

    # Iterate over the correlation matrix to identify pairs of descriptors with similar correlation
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) >= correlation_threshold_m:
                # Add correlated descriptors to the set
                merged_descriptors.add((correlation_matrix.columns[i], correlation_matrix.columns[j]))

    merged_data = data.copy()

    # Merge descriptors
    for descriptor_pair in merged_descriptors:
        # Check if both descriptors exist in the dataset
        if all(descriptor in merged_data.columns for descriptor in descriptor_pair):
            merged_descriptor_name = '_'.join(descriptor_pair)
            merged_data[merged_descriptor_name] = (data[descriptor_pair[0]] + data[descriptor_pair[1]]) / 2
            merged_data.drop(list(descriptor_pair), axis=1, inplace=True)
            
    X_m = merged_data.drop(columns=['target']) # Input features (molecular descriptors)
    Y_m = merged_data['target'] # Target variable (biological response)

    correlation_Y_m = X_m.apply(lambda x: x.corr(Y_m))
    selected_features_m = correlation_Y_m[correlation_Y_m >= correlation_threshold].index.tolist()

    selected_data_m = merged_data[selected_features_m]

    print("Selected Features from merged data:")
    print(selected_features_m)
    
    return pd.DataFrame(data=merged_data[selected_features_m + ['target']])

As you can see, the linear correlation is not too high between the target_value and the descriptors, so I have to experiment fith another feature selection methods.

#### Tree based selection

In [15]:
def tree_based_original(data, n_estimators=100, top_n=50, random_state=42):
    X = data.drop(columns=['target'])
    Y = data['target']
    
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
    rf.fit(X, Y)

    feature_importances = rf.feature_importances_
    importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    top_features = importance_df['Feature'].head(top_n).tolist()
    
    print("Top features:")
    top_features[:5]
    
    return pd.DataFrame(data=data[top_features + ['target']]) 

def tree_based_merged(data, n_estimators=100, top_n=50, random_state=42):
    correlation_matrix = data.corr()

    correlation_threshold = 0.5  # Adjust as needed

    merged_descriptors = set()

    # Iterate over the correlation matrix to identify pairs of descriptors with similar correlation
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) >= correlation_threshold:
                # Add correlated descriptors to the set
                merged_descriptors.add((correlation_matrix.columns[i], correlation_matrix.columns[j]))

    merged_data = data.copy()

    # Merge descriptors
    for descriptor_pair in merged_descriptors:
        # Check if both descriptors exist in the dataset
        if all(descriptor in merged_data.columns for descriptor in descriptor_pair):
            merged_descriptor_name = '_'.join(descriptor_pair)
            merged_data[merged_descriptor_name] = (data[descriptor_pair[0]] + data[descriptor_pair[1]]) / 2
            merged_data.drop(list(descriptor_pair), axis=1, inplace=True)
            
    X_m = merged_data.drop(columns=['target'])
    Y_m = merged_data['target']
    
    rf_m = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
    rf_m.fit(X_m, Y_m)

    feature_importances_m = rf_m.feature_importances_
    importance_df_m = pd.DataFrame({'Feature': X_m.columns, 'Importance': feature_importances_m})
    importance_df_m = importance_df_m.sort_values(by='Importance', ascending=False)
    top_features_m = importance_df_m['Feature'].head(top_n).tolist()
    
    print("Top featuresmerged:")
    top_features_m[:5]
    
    return pd.DataFrame(data=merged_data[top_features_m + ['target']])

#### PCA

In [16]:
def pca(data, n_components=50):
    X = data.drop(columns=['target'])
    y = data['target']
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    pca = PCA(n_components=n_components)

    pca.fit(X_scaled)

    X_pca = pca.transform(X_scaled)

    principal_components_df = pd.DataFrame(data=X_pca, columns=[f'PC{i+1}' for i in range(n_components)])

    return principal_components_df

Calling funcitons:

In [17]:
#correlation_selection_merged(data)
#tree_based_merged(data)
#pca(data, n_components = 10)

### Polynomial feature selection

In [18]:
# data_float32 = data.astype('float32')

In [19]:
# from sklearn.preprocessing import PolynomialFeatures

# # Feature Engineering
# poly = PolynomialFeatures(degree=1.5, interaction_only=True, include_bias=False)
# X_poly = poly.fit_transform(X)

# # Convert the polynomial feature matrix to a DataFrame
# X_poly_df = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(X.columns))

# # Concatenate the original features with the polynomial features
# X_combined = pd.concat([X, X_poly_df], axis=1)

This method occures memory problems.

## Summary of the feature selection
We created a feature selection according to linear correlation for original data, and tried to merge the descriptors with similar correlation value in relation with the target value. [selected_features, selected_features_m] 


We also selected features using tree based feature selection method using the original and the merged data. These lists represents the top 50 features. [top_features, top_features_m]


We tried polynomial feature selection as well, but in this case we had memory problems.

#### Selected features dataframe export functions:

You can get the dataframes, which are containing the specific selected features with calling: 

correlation_selection_original(data, correlation_threshold), correlation_selection_merged(data, correlation_threshold), tree_based_original(data, n_estimators, top_n, random_state), tree_based_merged(data, n_estimators, top_n, random_state), pca(data, n_components)